# Feature selection

## Exam question 23-24 (1st session)

Let us consider a regression task with $n=10$ inputs and one target.

The regression dataset is in the variable 
<span style="font-family:Courier; "> Q3_D </span> 
of the <span style="font-family:Courier; "> fsel1.pkl </span> file.

Note that the 11th column contains the target.


Consider a **wrapper backward selection** strategy where the learner is a locally linear regression algorithm which returns the prediction of a linear model fitted to the K nearest neighbours (K=10 and Euclidean distance), and the assessment is based on leave-one-out.

Return the index of the five most relevant features according to such feature selection strategy:

Use the instructions
```python
import pickle
with open("fsel1.pkl", 'rb') as f:
    data = pickle.load(f)
Q3_D=data["Q3_D"]
```
to load the <span style="font-family:Courier; "> Q3_D </span> variable in Python.

## Dataset extraction

In [3]:
import pickle
with open("fsel1.pkl", 'rb') as f:
    data = pickle.load(f)
Q3_D=data["Q3_D"]


## Local linear modelling functions

In [10]:

import numpy as np
from numpy.linalg import solve


## least-squares prediction in the point q 
## Linear model is trained with the input matrix X and the output vector Y
def lsq(X, Y, q):
    n = X.shape[1]
    N = X.shape[0]
    XX = np.column_stack((np.ones(N), X))
    beta = solve(XX.T @ XX, XX.T @ Y)
    yhat=np.concatenate((np.ones(1), q.flatten())) @ beta
    return float(yhat[0])


## Selection of the k nearest neighbour and least squares fitting
def LL(X, Y, q, k):
    N = X.shape[0]
    # Euclidean metric
    d = np.sqrt(np.sum(np.square(X - q), axis=1))
    index = np.argsort(d)[:k]
    LLhat=lsq(X[index], Y[index], q)
    return LLhat




## Wrapper backward selection

In [11]:
XY = Q3_D 
nn = XY.shape[1]
X = XY[:, :nn-1]
Y = XY[:, nn-1]

n = X.shape[1]
N = X.shape[0]
nfeat = 10
K = 10

fsub = list(range(n))

for ss in range(nfeat-1):
    Eloo = np.full(n, np.inf)
    for j in fsub:

        Eloo[j] = 0

        for i in range(N):
            Xi = np.delete(X, i, axis=0)
            Yi = np.delete(Y, i)        
            remaining_features = [f for f in fsub if f != j]
            Xi_subset = Xi[:, remaining_features]
            X_i_subset = X[i:i+1, remaining_features]
            Yhati = LL(Xi_subset, Yi.reshape(-1,1), X_i_subset, K)
            Eloo[j] += (Y[i] - Yhati)**2

        

        Eloo[j] = Eloo[j]/N

    
    min_index = np.argmin(Eloo)
    fsub.remove(min_index)
    print([f+1 for f in fsub])


[1, 2, 3, 4, 5, 6, 7, 9, 10]
[1, 3, 4, 5, 6, 7, 9, 10]
[1, 3, 4, 5, 6, 7, 9]
[1, 3, 4, 5, 7, 9]
[1, 3, 5, 7, 9]
[3, 5, 7, 9]
[5, 7, 9]
[5, 9]
[9]


## Exam question 22-23 (1st session)

Let us consider a regression task with $n=10$ inputs and one target whose dataset is in the variable <span style="font-family:Courier; "> D </span> 
of the  <span style="font-family:Courier; "> fsel2.pkl </span> file.


Note that the 11th column contains the target.

Consider a **wrapper forward selection** strategy where the learner is a 5NN (KNN with $K=5$ and Euclidean distance), and the assessment is based on leave-one-out.

Return the index of the five most relevant features according to this feature selection strategy

Use the instructions
```python
import pickle
with open("fsel2.pkl", 'rb') as f:
    data = pickle.load(f)
D=data["D"]
```
to load the <span style="font-family:Courier; "> D </span> variable in Python.


In [31]:
import pickle
with open("fsel2.pkl", 'rb') as f:
    data = pickle.load(f)
D=data["D"]

In [32]:
import numpy as np
import math # Import math for inf

# Define the KNN function
def KNN(X, Y, q, k):
    # X: training features (numpy array)
    # Y: training labels (numpy array)
    # q: query point (numpy array,  1xn)
    # k: number of neighbours
    N = X.shape[0]

    d = np.sqrt(np.sum(np.square(X - q), axis=1))

    index = np.argsort(d)[:k]
    return Y[index].mean()


In [34]:
n = 10 ## number of inputs
X = D[:, 0:n]
Y = D[:, n]

K = 5
N = len(Y)
n = X.shape[1]

fsub = []
nfeat = 5
for ss in range(nfeat):

   # Initialize Eloo (Leave-One-Out error) with infinity for each feature
    # Eloo is a numpy array of size n (number of columns in X)
    Eloo = np.full(n, np.inf)

    for j in range(n):
        if j in fsub:
            continue # Skip this feature if it's already selected

        Eloo[j] = 0

        for i in range(N):

            # Create the training data (Xi) and labels (Yi) by removing the i-th row/element
            # np.delete removes the specified index along the specified axis
            Xi = np.delete(X, i, axis=0)
            Yi = np.delete(Y, i)

            # Select the columns for the current subset of features (already selected fsub + candidate j)
            # fsub contains 0-based indices, j is the current 0-based index
            cols_subset = fsub + [j]

            # Training data subset for KNN: select rows from Xi and the columns in cols_subset
            Xi_subset = Xi[:, cols_subset]

            # Test point for KNN: select the i-th row from the original X and the columns in cols_subset
            # X[i, cols_subset] results in a 1D numpy array
            # .reshape(1, -1) reshapes it into a 1xM matrix as expected by the KNN function's 'q' parameter
            qi_subset = X[i, cols_subset].reshape(1, -1)

            # Perform KNN prediction for the left-out data point
            Yhati = KNN(Xi_subset, Yi, qi_subset, K)

           # Accumulate the squared error for the current candidate feature j
            # Y[i] is the actual label for the i-th data point (0-based index)
            Eloo[j] = Eloo[j] + (Y[i] - Yhati)**2

        # Calculate the mean squared error for the current candidate feature j
        Eloo[j] = Eloo[j] / N

    # Find the index (0-based) of the feature with the minimum mean squared error in Eloo
    best_j = np.argmin(Eloo)
    # Append the index of the best feature found in this iteration to the list of selected features
    fsub.append(best_j)

# Print the list of selected features (converting 0-based indices back to 1-based for output) and the value of K
print(f"bestfs={[x + 1 for x in fsub]} ")

bestfs=[2, 3, 1, 6, 8] 


## Exam question 22-23 (2nd session)

Let us consider a classification task with $n=10$ binary inputs and one binary target whose dataset is in the variable <span style="font-family:Courier; "> D </span> 
of the  <span style="font-family:Courier; "> fsel3.pkl </span> file.

Note that the 11th column contains the target.

Consider a **wrapper backward selection** strategy in which the learner is a Naive Bayes, and the assessment is based on the leave-one-out misclassification.


Use the instructions
```python
import pickle
with open("fsel3.pkl", 'rb') as f:
    data = pickle.load(f)
D=data["D"]
```
to load the <span style="font-family:Courier; "> D </span> variable in Python.


In [35]:
import pickle
with open("fsel3.pkl", 'rb') as f:
    data = pickle.load(f)
D=data["D"]

## Naive Bayes classifier

In [36]:

import numpy as np

def NB(X, Y, q):
    n = X.shape[1]
    N = len(Y)
    P0 = len(np.where(Y == 0)[0]) / N
    P1 = 1 - P0
    
    for i in range(n):
        I = np.where(X[:, i] == q[0, i])[0]
        I0 = np.where(Y[I] == 0)[0]
        I1 = np.where(Y[I] == 1)[0]
        P0 = P0 * len(I0) / len(I)
        P1 = P1 * len(I1) / len(I)
    
    return 0 if P0 > P1 else 1



In [37]:

N = D.shape[0]  
n = D.shape[1] - 1
    
X = D[:, :n]
Y = D[:, n]
    
    
Fset = np.arange(n) ## initial feature set

for r in range(n-5):
    Miscl = np.full(n, np.inf)

    for j in range(n):
        if j in Fset:
            Ehat = 0
            for i in range(N):
                remaining_features = np.setdiff1d(Fset, j)
                X_subset = X[np.arange(N) != i][:, remaining_features]
                Y_subset = Y[np.arange(N) != i]
                q = X[i:i+1, remaining_features]
                Yhat = NB(X_subset, Y_subset, q)
                Ehat += float(Yhat != Y[i])

            Miscl[j] = Ehat/N

    # Remove the feature that minimizes misclassification error
    Fset = np.setdiff1d(Fset, np.argmin(Miscl))

print(Fset+1)


[ 1  2  3  5 10]


## LIN.QA.1

Let us consider a regression task with $n=10$ inputs $ {\mathbf x}_1, \dots {\mathbf x}_{10} $ and 1 output $ {\mathbf y} $.

Let the training set be in the variable <span style="font-family:Courier; ">D</span> in the pickle file <span style="font-family:Courier; ">data1.pkl</span>, where the first 10 columns contain the inputs and the last column contains the output.

Suppose we want to perform an **embedded feature selection** by ridge regression where the set of values of the shrinkage parameter $ \lambda $ are contained in the Python variable <span style="font-family:Courier; "> lam</span>.

By using Python
* compute the optimal $ \lambda $ shrinkage parameter for a ridge-regression approach by using a leave-one-out assessment strategy 
* for the optimal $ \lambda $  computed above,  return the three most relevant features 


Note that you should use only basic Python functions.








 

 

 

In [30]:
import pickle
with open("data1.pkl", 'rb') as f:
    data = pickle.load(f)
D=data["D"].to_numpy()
lam=data["lam"].to_numpy()

D.shape

(200, 11)

In [31]:
import numpy as np

n = 10

X = D[:, :n]
Y = D[:, n]
N = len(Y)

XX = np.c_[np.ones(N), X]

p = len(lam)

Eloo = np.zeros(p)

for j in range(p):
    for i in range(N):
        ## leave-one-out
        Xi = XX[np.arange(N) != i, :]
        Yi = Y[np.arange(N) != i]
        betai = np.linalg.solve(Xi.T @ Xi + lam[j] * np.eye(n + 1), Xi.T @ Yi)
        Yhati = XX[i, :] @ betai
        Eloo[j] += (Y[i] - Yhati)**2
    Eloo[j] = Eloo[j] / N

bestlambda = lam[np.argmin(Eloo)]

## computation of the beta coefficients for the optimal lambda
beta = np.linalg.solve(XX.T @ XX + bestlambda * np.eye(n + 1), XX.T @ Y)


## retrieval of the three most relevant features
feature_coeffs_abs = np.abs(beta[1:n+1])
top_3_indices_0based = np.argsort(-feature_coeffs_abs)[0:3]
top_3_feature_indices_1based = top_3_indices_0based + 1

print(f"bestlambda={bestlambda} fs={list(top_3_feature_indices_1based)}")




bestlambda=[0.2] fs=[2, 3, 1]
